### Conversiones de Datos

Muchas veces vamos a tener que leer datos desde alguna fuente que no tiene el formato de fecha, numerico o del tipo que necesitamos.

El caso mas simple es cuando necesitamos ingresar un numero desde teclado. Podemos usar la funcion `Console.ReadLine()` pero la misma nos retorna un string, no un valor numerico. Mas problemas podemos llegar a tener si el valor numerico tiene que ser con decimales o notacion cientifica: el separador de decimales es el punto o la coma? se aceptan numeros negativos? 

Para estos casos la libreria de clases de .NET dispone de funciones que permiten convertir una cadena en un tipo especifico de dato.

Las funciones que vamos a encontrar *en los tipos que admiten ser convertidos desde string*:

- Parse
- TryParse
- ParseExact (tipos de fecha/hora)
- TryParseExact (tipos de fecha/hora)

Si se encuentran, estas funciones son **metodos de clase**. Esto es razonable porque en realidad queremos obtener una instancia del tipo a partir de un string, y en el caso que fueran metodos de instancia necesitaria tene la instancia previamente creada...seria una paradoja. 

Dijimos anteriormente que no todos los tipos pueden convertirse desde string. En la libreria de clases .NET vamos a encontrar al menos los siguientes:

- todos los tipos numericos enteros (int, long, etc)
- los tipos char y bool
- los tipos float y double
- el tipo decimal
- los tipos DateTime, TimeSpan, DateOnly y TimeOnly

Hay varios mas, y nosotros podemos definir los nuestros, extendiendo interfaces como `IParseable<T>` o `INumber<T>` 

Veamos un ejemplo del metodo `Parse`, supongamos que necesitamos ingresar por teclado un numero decimal, el contenido de lo que el usuario ingresa esta en la variable input:


In [ ]:
string input = "123.456";

decimal valorLeido;

valorLeido = decimal.Parse(input);    // OK

Console.WriteLine(valorLeido);  

input = "123AB";

valorLeido = decimal.Parse(input);    // Error de formato

Console.WriteLine(valorLeido);  

input = "-123.45e+54";

valorLeido = decimal.Parse(input);    // Error de formato

Console.WriteLine(valorLeido);  



Si en cambio usaramos un **double** para el ultimo intento...

In [ ]:
//  los contenidos de las variables se conservan entre bloques en los Notebooks
//  por esa razon no es necesario redefinir input
//
double valorLeido1 = double.Parse(input);

Console.WriteLine(valorLeido1);  


Pudimos observar que Parse genera una excepcion (error) si se presenta por ejemplo un problema de formato.

La manera de evitar esto es usar `TryParse` que retorna como resultado un bool que sera true si la conversion tuvo exito y false si fallo, pero no genera un corte del programa. Lo que tiene como desventaja es que el valor retornado, en caso de exito, se guarda en un parametro **out** declarado dentro de la funcion. Por esa razon debemos ya tener una variable declarada del tipo correcto para poder usar TryParse. Esto ocurria igual con Parse, o sea la variable ya tenia que estar declarada, solo que en este caso el nombre de variable se pasa como argumento a la funcion.

Probemos los mismos casos:

In [ ]:
string input = "123.456";

decimal valorLeido;

if (decimal.TryParse(input, out valorLeido))
  Console.WriteLine(valorLeido);  
else
  Console.WriteLine("Error de conversion");

input = "123AB";

if (decimal.TryParse(input, out valorLeido))
  Console.WriteLine(valorLeido);  
else
  Console.WriteLine("Error de conversion");

input = "-123.45e+54";

if (decimal.TryParse(input, out valorLeido))
  Console.WriteLine(valorLeido);  
else
  Console.WriteLine("Error de conversion");

double valorLeido1;

if (double.TryParse(input, out valorLeido1))
  Console.WriteLine(valorLeido1);  
else
  Console.WriteLine("Error de conversion");



##### Parse con varias opciones

En los tipos como fechas, los formatos que existen son variadisimos y pueden cambiar en el momento menos esperado. 

Supongamos que tenemos una cadena con una fecha y supongamos que representa el 31/12/2023, podria ser que alguien ingrese como fecha 31/12 o 31-12-2023 o 31/12/23...etc

Usando la funcion `TryParseExact` podemos pasar un arreglo de cadenas posibles de formato para que ls funcion chequee contra todas ellas

In [ ]:

//  string fechaTexto = "31/10/23";
//  string fechaTexto = "31/10";
string fechaTexto = "31-07-2023";

DateTime fecha;

if (DateTime.TryParseExact(fechaTexto, new[] {"dd-MM-yyyy", "dd/MM", "dd/MM/yy"}, null, 
   System.Globalization.DateTimeStyles.None, out fecha))
  fecha.Display();
else
  Console.WriteLine("Formato invalido");




Esto por supuesto funciona pero tengamos en cuenta que se validan estos formatos y no otros que tal vez sean tambien coherentes como por ejemplo el de poner un solo digito en el mes o dia que sean menores a 10.

Todas estas opciones deberian chequearse si queremos una aplicacion consistente y robusta

##### Conversiones y Culturas

Cuando involucramos la cultura en el parsing de cadenas sumamos otro problema mas. Dependiendo la cultura de la UI o del sistema donde estemos ejecutando nuestra aplicacion, las funciones standard de parsing pueden dar errores.

Con el `TryParseExact` podemos superar parte de estos problemas, por ejemplo en el caso siguiente la fecha es reconocida para ambas culturas (es-AR u en-US) pero tampoco podemos relajar tanto las validaciones porque podriamos realmente estar referenciando una fecha equivocada, o un error de tipeo.

La demo siguiente es para ilustrar el hecho, no significa que deba hacerse!!

Los tipos numericos tambien estan influidos por la cultura, en nuestro caso el punto/coma decimal cambia si estamos en es-AR o en en-US. Si tenemos el VS Code en español, entonces en el siguiente ejemplo veremos como un numero "12.3454" (en formato US que es muy comun en las salidas de bases de datos) es convertido incorrectamente, mientras que si especificamos dentro de las opciones que use la cultura invariante, funciona OK!

Todos los tipos tienen una posibilidad de convertirse a string mediante el metodo ToString(), este metodo es llamado automaticamente cuando usamos por ejemplo Console.WriteLine() pero nada nos impide que podamos invocarlo explicitamente

Algunas versiones de ToString() permiten especificar un formato adicional (en general las numericas y tipo fecha)

In [ ]:
string fechaTexto = "12-31-2023"; //  probar 31-12-2023

DateTime fecha;

System.Globalization.CultureInfo.CurrentCulture.Name.Display();

if (DateTime.TryParseExact(fechaTexto, new[] {"dd-MM-yyyy", "MM-dd-yyyy"}, null, 
   System.Globalization.DateTimeStyles.None, out fecha))
  fecha.Display();
else
  Console.WriteLine("Formato invalido");

string numero = "12.3454";

Double.TryParse(numero, System.Globalization.NumberStyles.Any, null, out var valor);
//  Double.TryParse(numero, System.Globalization.NumberStyles.Any, System.Globalization.CultureInfo.InvariantCulture, out var valor);

valor.ToString("#.####").Display();  

double pi = Math.PI;

pi.ToString("#.##").Display();

DateTime.Now.ToString("dd MMM yy").Display();
